In [1]:
from itertools import islice, cycle
from tau_profile_parser import TauProfileParser
import pandas as pd
import plotly.graph_objs as go
import plotly as py
import plotly.express as px

Turn on offline mode

In [2]:
py.offline.init_notebook_mode(connected=True)

Plotly Bar Plot Examples
========================

This notebook demonstrates using the TauProfileParser to parse TAU profile.x.y.z files and using Plotly to generate ParaProf-style bar plots from them.

The TAU Profile Parser
----------------------

First, we call `TauProfileParser.parse` on the path to a directory containing TAU profile files. 

Note that the parser does not currently have any native support for MULTI_ directories, so if you have those
you'll have to call parse multiple times and then, if you wish, merge the resulting dataframes. This functionality should be added to the parser.

In [3]:
lulesh_data = TauProfileParser.parse('lulesh_profile')


This returns an object that provides access to the metric name, metadata, and the interval and atomic/userevent data.

The `metric` attribute contains the metric that values in this profile represent. (This representation will need to change when multi-metric data can be read in directly)


In [4]:
lulesh_data.metric

b'PAPI_TOT_INS, PAPI_FP_OPS, TIME, PAPI_TOT_CYC'

The `metadata` attribute contains a Python dictionary of the metadata recorded in the profile.0.0.0 file

In [5]:
lulesh_data.metadata

{'Metric Name': 'PAPI_TOT_INS, PAPI_FP_OPS, TIME, PAPI_TOT_CYC',
 'CPU MHz': '3783.000000MHz',
 'CPU Type': 'POWER9, altivec supported',
 'CWD': '/storage/users/MTrappett/tau2',
 'Command Line': '../LULESH//lulesh2.0 -p -i 200',
 'Ending Timestamp': '1607457840198605',
 'Executable': '/storage/users/MTrappett/LULESH/lulesh2.0',
 'Hostname': 'gorgon',
 'Local Time': '2020-12-08T12:03:37-08:00',
 'Memory Size': '395901568 kB',
 'Node Name': 'gorgon',
 'OMP_CHUNK_SIZE': '0',
 'OMP_DYNAMIC': 'FALSE',
 'OMP_MAX_ACTIVE_LEVELS': '2147483647',
 'OMP_MAX_THREADS': '8',
 'OMP_NUM_PROCS': '160',
 'OMP_SCHEDULE': 'STATIC',
 'OMP_THREAD_LIMIT': '2147483647',
 'OS Machine': 'ppc64le',
 'OS Name': 'Linux',
 'OS Release': '4.14.0-115.18.1.el7a.ppc64le',
 'OS Version': '#1 SMP Wed Jan 29 11:49:09 EST 2020',
 'OpenMP Version': '3.1',
 'OpenMP Version String': '201107',
 'Starting Timestamp': '1607457817329141',
 'TAU Architecture': 'default',
 'TAU Config': ' -ompt-openmp -bfd=download -papi=/usr/local/

The `indices` attribute contains the (node, context, thread) tuples that are present in this profile.

In [6]:
lulesh_data.indices

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7)]

The `atomic_data()` function returns a Pandas dataframe containing the atomic events (user and context events). This is a multi-index dataframe with Node, Context, Thread, and Timer as the indices.

(This profile was collected with an instrumented version of Open MPI which collects memory allocations for different object sizes.)

In [7]:
lulesh_data.atomic_data()

,,,,Count,Maximum,Minimum,Mean,SumSq
Node,Context,Thread,Timer,,,,,


The `interval_data()` function returns the timer data. As with atomic_data(), it is returned as a multi-index Pandas dataframe with Node, Context, Thread, and Timer as the indices.

In [8]:
lulesh_data.interval_data()

Intervals                                                              Calls  \
Metric                                                                         
Node Context Thread Timer                                                      
0    0       0      .TAU application                                       1   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...     1   
                    .TAU application => _GLOBAL__sub_I__Z14CalcElem...     1   
                    __static_initialization_and_destruction_0(int, ...     1   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...     1   
...                                                                      ...   
             7      CalcElemShapeFunctionDerivatives(double const*,...  1251   
                    CalcElemShapeFunctionDerivatives(double const*,...  1250   
                    CalcElemShapeFunctionDerivatives(double const*,...  1250   
                    SumElemStressesToNodeForces(double const (*) [8...  1256   
                    CalcElemVelocityGradient(double const*, double ...  1256   

Intervals                                                              Subcalls  \
Metric                                                                            
Node Context Thread Timer                                                         
0    0       0      .TAU application                                          3   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...        1   
                    .TAU application => _GLOBAL__sub_I__Z14CalcElem...        1   
                    __static_initialization_and_destruction_0(int, ...        0   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...        0   
...                                                                         ...   
             7      CalcElemShapeFunctionDerivatives(double const*,...        0   
                    CalcElemShapeFunctionDerivatives(double const*,...     1250   
                    CalcElemShapeFunctionDerivatives(double const*,...        0   
                    SumElemStressesToNodeForces(double const (*) [8...        0   
                    CalcElemVelocityGradient(double const*, double ...        0   

Intervals                                                              ProfileCalls  \
Metric                                                                                
Node Context Thread Timer                                                             
0    0       0      .TAU application                                              0   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...            0   
                    .TAU application => _GLOBAL__sub_I__Z14CalcElem...            0   
                    __static_initialization_and_destruction_0(int, ...            0   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{/h...            0   
...                                                                             ...   
             7      CalcElemShapeFunctionDerivatives(double const*,...            0   
                    CalcElemShapeFunctionDerivatives(double const*,...            0   
                    CalcElemShapeFunctionDerivatives(double const*,...            0   
                    SumElemStressesToNodeForces(double const (*) [8...            0   
                    CalcElemVelocityGradient(double const*, double ...            0   

Intervals                                                                                          Group  \
Metric                                                                                                     
Node Context Thread Timer                                                                                  
0    0       0      .TAU application                                                 GROUP="TAU_DEFAULT"   
                    _GLOBAL__sub_I__Z14CalcElemVolumePKdS0_S0_ [{

We can manipulate this using normal Pandas functions. For example, here are the timers on (0,0,0) sorted by number of calls descending.

In [9]:
lulesh_data.interval_data().loc[0,0,0].sort_values('Calls', ascending=False)

Intervals                                            Calls Subcalls  \
Metric                                                                
Timer                                                                 
CalcElemCharacteristicLength(double const*, dou...  121663    99990   
OpenMP_Implicit_Task => CalcElemCharacteristicL...  121663    99990   
OpenMP_Implicit_Task => CalcElemVolumeDerivativ...  120892    99962   
CalcElemVolumeDerivative(double*, double*, doub...  120892    99962   
CalcElemNodeNormals(double*, double*, double*, ...  119578    99990   
...                                                    ...      ...   
Domain::Domain(int, int, int, int, int, int, in...       1        0   
Domain::SetupElementConnectivities(int) [{/home...       1        0   
Domain::Domain(int, int, int, int, int, int, in...       1        0   
Domain::SetupBoundaryConditions(int) [{/home/us...       1        0   
main [{/home/users/MTrappett/LULESH/lulesh.cc} ...       1        0   

Intervals                                          ProfileCalls  \
Metric                                                            
Timer                                                             
CalcElemCharacteristicLength(double const*, dou...            0   
OpenMP_Implicit_Task => CalcElemCharacteristicL...            0   
OpenMP_Implicit_Task => CalcElemVolumeDerivativ...            0   
CalcElemVolumeDerivative(double*, double*, doub...            0   
CalcElemNodeNormals(double*, double*, double*, ...            0   
...                                                         ...   
Domain::Domain(int, int, int, int, int, int, in...            0   
Domain::SetupElementConnectivities(int) [{/home...            0   
Domain::Domain(int, int, int, int, int, int, in...            0   
Domain::SetupBoundaryConditions(int) [{/home/us...            0   
main [{/home/users/MTrappett/LULESH/lulesh.cc} ...            0   

Intervals                                                                      Group  \
Metric                                                                                 
Timer                                                                                  
CalcElemCharacteristicLength(double const*, dou...               GROUP="TAU_DISABLE"   
OpenMP_Implicit_Task => CalcElemCharacteristicL...  GROUP="TAU_CALLPATH|TAU_DEFAULT"   
OpenMP_Implicit_Task => CalcElemVolumeDerivativ...  GROUP="TAU_CALLPATH|TAU_DEFAULT"   
CalcElemVolumeDerivative(double*, double*, doub...               GROUP="TAU_DISABLE"   
CalcElemNodeNormals(double*, double*, double*, ...               GROUP="TAU_DISABLE"   
...                                                                              ...   
Domain::Domain(int, int, int, int, int, int, in...  GROUP="TAU_CALLPATH|TAU_DEFAULT"   
Domain::SetupElementConnectivities(int) [{/home...               GROUP="TAU_DEFAULT"   
Domain::Domain(int, int, int, int, int, int, in...  GROUP="TAU_CALLPATH|TAU_DEFAULT"   
Domain::SetupBoundaryConditions(int) [{/home/us...               GROUP="TAU_DEFAULT"   
main [{/home/users/MTrappett/LULESH/lulesh.cc} ...  GROUP="TAU_CALLPATH|TAU_DEFAULT"   

Intervals                                                                                  Timer Name  \
Metric                                                                                                  
Timer                                                                                                   
CalcElemCharacteristicLength(double const*, dou...  CalcElemCharacteristicLength(double const*, do...   
OpenMP_Implicit_Task => CalcElemCharacteristicL...  OpenMP_Implicit_Task => CalcElemCharacteristic...   
OpenMP_Implicit_Task => CalcElemVolumeDerivativ...  OpenMP_Implicit_Task => CalcElemVolumeDerivati...   
CalcElemVolumeDerivative(double*, double*, doub...  CalcElemVolumeDerivative(double*, double*, dou...   
CalcElemNodeNormals(double*, double*, double*, ...  CalcElemNodeNormals(double*, double*, double*,

## Sunburst chart

The data to create a sunburst chart is within the timers that are in TAU_CALLPATH group. These timers specify callpaths, where calls are seperated by ' => '. We create a new dataframe. We split them by ' => ' and add a new column for each call in the dataframe. Since the depth of the call path might be different for each timer, we expand dataframe as deep as the deepest callpath, and assing "" for empty calls.

In [10]:
node, context, thread = 0, 0, 0
col_name =('Exclusive','TIME')
TOP_N = 10

In [11]:
data = lulesh_data.interval_data()

#keeping only the call-path names
data = data.loc[data["Group"].str.contains("TAU_CALLPATH")]         
data = data.loc[node, context, thread][[col_name]] 
sorted_data = data.sort_values(by=[col_name], ascending=False)

# everything below the 'TOP_N'
others = pd.DataFrame(sorted_data[TOP_N:].sum()).transpose() # the values below top N
truncated = pd.concat([sorted_data[:TOP_N], others], copy=False)

# adding the index, which is the callpath, as a column
truncated['Timer'] = truncated.index            

# seperating the calls in the callpath, adding the seperated calls back to main df
tmp = truncated['Timer'].str.split("  => ",expand=True) 
truncated[list(tmp.columns)] = tmp 
truncated = truncated.drop(columns=["Timer"]) # we no longer need the callpath timer

truncated = truncated.append(pd.Series(name='Others'))
truncated = truncated.fillna("")
others_val = others[col_name].values[0]
truncated.loc['Others', col_name] = others_val

sunburst_data = truncated
sunburst_data

/home/users/MTrappett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



Intervals,Exclusive,0
Metric,TIME,
"OpenMP_Parallel_Region CalcKinematicsForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1510, 0}] => OpenMP_Implicit_Task",4.75683e+06,OpenMP_Parallel_Region CalcKinematicsForElems ...
"OpenMP_Parallel_Region IntegrateStressForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {521, 0}] => OpenMP_Implicit_Task",3.55422e+06,OpenMP_Parallel_Region IntegrateStressForElems...
"OpenMP_Parallel_Region CalcHourglassControlForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1009, 0}] => OpenMP_Implicit_Task",2.05636e+06,OpenMP_Parallel_Region CalcHourglassControlFor...
"OpenMP_Parallel_Region CalcFBHourglassForceForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {782, 0}] => OpenMP_Implicit_Task",1.24134e+06,OpenMP_Parallel_Region CalcFBHourglassForceFor...
"OpenMP_Implicit_Task => CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}]",1.12007e+06,OpenMP_Implicit_Task => CalcElemVolumeDerivati...
"OpenMP_Implicit_Task => CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}]",1.10308e+06,OpenMP_Implicit_Task => CalcElemCharacteristic...
"OpenMP_Implicit_Task => CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {426,0}]",1.09387e+06,OpenMP_Implicit_Task => CalcElemNodeNormals(do...
"OpenMP_Implicit_Task => CalcElemVolume(double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {1362,0}]",790193,OpenMP_Implicit_Task => CalcElemVolume(double ...
"EvalEOSForElems(Domain&, double*, int, int*, int) [{/home/users/MTrappett/LULESH/lulesh.cc} {2207,0}] => CalcEnergyForElems(double*, double*, double*, double*, double*, double*, double*, double*, double*, double*, double*, double*, double*, double, double, double, double, double, double*, double*, double, double, int, int*) [{/home/users/MTrappett/LULESH/lulesh.cc} {2059,0}]",585395,"EvalEOSForElems(Domain&, double*, int, int*, i..."


In [12]:
# we also keep track of each individual call(not in TAU_CALLPATH)
timer_data = lulesh_data.interval_data().loc[node, context, thread].sort_values('Calls', ascending=False)
timer_data = timer_data.loc[~timer_data["Group"].str.contains("TAU_CALLPATH")].loc[:,[col_name]]

# some indices have trailing white spaces, we remove the white spaces
timer_data.index = timer_data.index.map(str.strip)
timer_data

Intervals,Exclusive
Metric,TIME
Timer,
"CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}] [THROTTLED]",1103082
"CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}] [THROTTLED]",1120066
"CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {426,0}] [THROTTLED]",1093873
"CalcElemVolume(double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {1362,0}] [THROTTLED]",1053489
"CalcElemVolume(double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1286,0}] [THROTTLED]",135828
...,...
"Domain::SetupThreadSupportStructures() [{/home/users/MTrappett/LULESH/lulesh-init.cc} {273,0}]",936
"Domain::CreateRegionIndexSets(int, int) [{/home/users/MTrappett/LULESH/lulesh-init.cc} {402,0}]",392


In [13]:
levels = []
for i in sunburst_data.columns[1:]:
    level = truncated.loc[:,i].values.tolist()
    level = [l.strip() for l in level]
    levels.append(level)
levels

[['OpenMP_Parallel_Region CalcKinematicsForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1510, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region IntegrateStressForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {521, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region CalcHourglassControlForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1009, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region CalcFBHourglassForceForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {782, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Implicit_Task => CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}]',
  'OpenMP_Implicit_Task => CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}]',
  'OpenMP_Implicit_Task => CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTra

In [14]:
def remove_white_space(level):
    l = level.copy()
    i = 0
    while i in range(len(l)):
        if l[i] == '':
            l.pop(i)
        else:
            i += 1
    return l

for i in range(len(levels)):
    levels[i] = remove_white_space(levels[i])
levels

[['OpenMP_Parallel_Region CalcKinematicsForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1510, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region IntegrateStressForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {521, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region CalcHourglassControlForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1009, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Parallel_Region CalcFBHourglassForceForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {782, 0}] => OpenMP_Implicit_Task',
  'OpenMP_Implicit_Task => CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}]',
  'OpenMP_Implicit_Task => CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}]',
  'OpenMP_Implicit_Task => CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTra

'go.Sunburst', the plotly module that we're using, expects 2 lists of parents and labels. These lists have to have the same length, labels list has to contain every label that we're going to show in out sunburst chart, and parents list contains the label of the parent of each label. If a label doesn't have a parent, then we need to assign a ''(empty string) as it's parent.  
for example:  
labels = ['a', 'b', 'c', 'd']  
parents = ['', 'a', 'a', 'b']  
here a is the root, and has children b and c; and b has d as it's child

In [15]:
def make_parents_and_labels(args):
    parents = []
    labels = []
    argc = len(args)
    for i in range(argc):
        level = args[i]
        for j in range(len(level)):
            if i == 0: # first level (root), no parent
                parents.append('')
            
            else:
                parent = args[i-1]
                parents.append(parent[j])
                
            labels.append(level[j])
                
    return parents, labels

parents, labels = make_parents_and_labels(levels)
print(parents,'\n\n')
labels

['', '', '', '', '', '', '', '', '', ''] 




['OpenMP_Parallel_Region CalcKinematicsForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1510, 0}] => OpenMP_Implicit_Task',
 'OpenMP_Parallel_Region IntegrateStressForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {521, 0}] => OpenMP_Implicit_Task',
 'OpenMP_Parallel_Region CalcHourglassControlForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1009, 0}] => OpenMP_Implicit_Task',
 'OpenMP_Parallel_Region CalcFBHourglassForceForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {782, 0}] => OpenMP_Implicit_Task',
 'OpenMP_Implicit_Task => CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}]',
 'OpenMP_Implicit_Task => CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}]',
 'OpenMP_Implicit_Task => CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/L

Plotly doesn't expect duplicates, if we are giving it duplicates, we have to also give each duplicate an id. 
Plotly is a little finicky with these ids. Empirically I have found that the ids have to have the same name as the labels, but the duplicate labels, have to have a unique extension of sorts to the original name of the label for their ids.  For example if in our lables list we have ["OpenMP_Implicit_Task", "OpenMP_Implicit_Task"], their ids need to be in this form ["OpenMP_Implicit_Task", "OpenMP_Implicit_Task - < unique-extension >"]

In [16]:
def find_dups(l):
    m = {}
    ids = []
    for i in l:
        if i in m:
            m[i] += 1
            ids.append('{} - {}th'.format(i, m[i]))
        else:
            if i != "":
                m[i] = 1
            ids.append(i)
    return ids

ids = []
for level in levels:
    ids += level       # initially we give the ids the same name as labels
    
ids = find_dups(ids)   # then we assign unique id to each duplicate

In [23]:
values = [] # not getting populated
print(timer_data.index) # look through this why not in timer_data.index
print('labels: ', labels)
for i in labels: 
    if i in timer_data.index:
        values.append(timer_data.loc[i, col_name])
    else:
        print("no such index {}".format(i))

Index(['CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}] [THROTTLED]',
       'CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}] [THROTTLED]',
       'CalcElemNodeNormals(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {426,0}] [THROTTLED]',
       'CalcElemVolume(double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {1362,0}] [THROTTLED]',
       'CalcElemVolume(double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1286,0}] [THROTTLED]',
       'SumElemStressesToNodeForces(double const (*) [8], double, double, double, double*, dou

Now appending the "Others" value.

In [22]:
def add_others(ids, labels, values, parents):
    def copy(ids, labels, values, parents):
        return (ids.copy(), labels.copy(), values.copy(), parents.copy())
    
    ids, labels, values, parents = copy(ids, labels, values, parents) # values not being copied, only the 'Others' value
    ids.append('Others'), 
    labels.append('Others'), 
    values.append(sunburst_data.loc['Others', col_name]), 
    parents.append('')
    return ids, labels, values, parents

ids, labels, values, parents = add_others(ids, labels, values, parents)
print('ids', len(ids),'\n labels: ', len(labels), '\n values: ', len(values), '\n parents: ', len(parents))
print('ids',ids,'\n labels: ', labels, '\n values: ', values, '\n parents: ', parents)
values

values:  [5376291, 5376291, 5376291]
ids 14 
 labels:  14 
 values:  4 
 parents:  14
ids ['OpenMP_Parallel_Region CalcKinematicsForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1510, 0}] => OpenMP_Implicit_Task', 'OpenMP_Parallel_Region IntegrateStressForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {521, 0}] => OpenMP_Implicit_Task', 'OpenMP_Parallel_Region CalcHourglassControlForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {1009, 0}] => OpenMP_Implicit_Task', 'OpenMP_Parallel_Region CalcFBHourglassForceForElems [{/home/users/MTrappett/LULESH/lulesh.cc} {782, 0}] => OpenMP_Implicit_Task', 'OpenMP_Implicit_Task => CalcElemVolumeDerivative(double*, double*, double*, double const*, double const*, double const*) [{/home/users/MTrappett/LULESH/lulesh.cc} {630,0}]', 'OpenMP_Implicit_Task => CalcElemCharacteristicLength(double const*, double const*, double const*, double) [{/home/users/MTrappett/LULESH/lulesh.cc} {1399,0}]', 'OpenMP_Implicit_Task => CalcElemNodeNormals(double*, doub

[5376291, 5376291, 5376291, 5376291]

Finally, before drawing the chart, we need to make sure the length of these lists are all the same.

In [19]:
assert(len(values) == len(ids) == len(labels) == len(parents))

AssertionError: 

In [ ]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    ids=ids,
    labels=labels,
    parents=parents,
    values=values,
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()